In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Titanic/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Titanic/gender_submission.csv')

In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
train.set_index('PassengerId', inplace=True)
test.set_index('PassengerId', inplace=True)

In [ ]:
all_data = pd.concat([train, test], sort=False)
all_data_index = all_data.index

In [ ]:
na_check = all_data.isna().sum() / len(all_data)
na_check = na_check[na_check > 0]
na_check

Survived    0.319328
Age         0.200917
Fare        0.000764
Cabin       0.774637
Embarked    0.001528
dtype: float64

In [ ]:
all_data = all_data.drop(['Cabin'], axis=1)
all_data = all_data.drop(['Ticket'], axis=1)

In [ ]:
all_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [ ]:
col = all_data.columns

for i in col:
  nq = all_data[i].unique()

  print(i)
  print(nq)

Survived
[ 0.  1. nan]
Pclass
[3 1 2]
Name
['Braund, Mr. Owen Harris'
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'
 'Heikkinen, Miss. Laina' ... 'Saether, Mr. Simon Sivertsen'
 'Ware, Mr. Frederick' 'Peter, Master. Michael J']
Sex
['male' 'female']
Age
[22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.   22.5  18.5  67.   76.   26.5  60.5  11.5
  0.33  0.17 38.5 ]
SibSp
[1 0 3 4 2 5 8]
Parch
[0 1 2 5 3 4 6 9]
Fare
[  7.25    71.2833   7.925   53.1      8.05     8.4583  51.8625  21.075
  11.1333  30.0708  16.7 

In [ ]:
title = ['Mr.', 'Miss.', 'Mrs.', 'Master.', 'Dr.', 'Ms.', 'Col.', 'Capt.', 'Rev.', 'Major']

all_data['Name_None'] = False
for i in title:
  all_data['Name_'+i] = False

for idx, row in all_data.iterrows():
  name = row['Name']
  name_ = 'None'
  name = name.replace('Mr ', 'Mr.')
  name = name.replace('Mrs ', 'Mrs.')

  for tt in title:
    if name.find(tt) != -1:
      name_ = tt
 
  all_data['Name_'+name_][idx] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
all_data = all_data.drop(['Name'], axis=1)

In [ ]:
data_object = all_data.select_dtypes(include=object)
data_non_object = all_data.select_dtypes(exclude=object)

In [ ]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(data_non_object)
data_non_object_ = imputer.transform(data_non_object)
data_non_object_ = pd.DataFrame(data_non_object_, columns=data_non_object.columns, index=all_data_index)
data_non_object_

,Survived,Pclass,Age,SibSp,Parch,Fare,Name_None,Name_Mr.,Name_Miss.,Name_Mrs.,Name_Master.,Name_Dr.,Name_Ms.,Name_Col.,Name_Capt.,Name_Rev.,Name_Major
PassengerId,,,,,,,,,,,,,,,,,
1,0.000000,3.0,22.000000,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,1.0,38.000000,1.0,0.0,71.2833,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000000,3.0,26.000000,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,1.0,35.000000,1.0,0.0,53.1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,3.0,35.000000,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0.383838,3.0,29.881138,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1306,0.383838,1.0,39.000000,0.0,0.0,108.9000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1307,0.383838,3.0,38.500000,0.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
all_data = pd.merge(data_object, data_non_object_, left_index=True, right_index=True)

In [ ]:
all_data.head()

,Sex,Embarked,Survived,Pclass,Age,SibSp,Parch,Fare,Name_None,Name_Mr.,Name_Miss.,Name_Mrs.,Name_Master.,Name_Dr.,Name_Ms.,Name_Col.,Name_Capt.,Name_Rev.,Name_Major
PassengerId,,,,,,,,,,,,,,,,,,,
1,male,S,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,female,C,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,female,S,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,female,S,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,male,S,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
all_data['Sex(status)'] = all_data['Sex'] == 'male'
all_data = all_data.drop(['Sex'], axis=1)

In [ ]:
all_data['Embarked(S)'] = (all_data['Embarked'] == 'S') | (all_data['Embarked'].isna())
all_data['Embarked(C)'] = all_data['Embarked'] == 'C'
all_data['Embarked(Q)'] = all_data['Embarked'] == 'Q'
all_data = all_data.drop(['Embarked'], axis=1)

In [ ]:
all_data['Family_members'] = 0
all_data['Family_members'] = 1 + all_data['SibSp'] + all_data['Parch']

In [ ]:
all_data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Name_None,Name_Mr.,Name_Miss.,Name_Mrs.,Name_Master.,Name_Dr.,Name_Ms.,Name_Col.,Name_Capt.,Name_Rev.,Name_Major,Sex_Male,Sex_Female,Embarked(S),Embarked(C),Embarked(Q),Family_members
PassengerId,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,True,False,False,2.0
2,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False,True,False,2.0
3,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True,False,False,1.0
4,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,True,False,False,2.0
5,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,True,False,False,1.0


In [ ]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [ ]:
cormat = train.corr()
cormat = cormat.index[abs(cormat['Survived']) > 0.1]
cormat

Index(['Survived', 'Pclass', 'Fare', 'Name_Mr.', 'Name_Miss.', 'Name_Mrs.',
       'Sex(status)', 'Embarked(S)', 'Embarked(C)'],
      dtype='object')

In [ ]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,Name_None,Name_Mr.,Name_Miss.,Name_Mrs.,Name_Master.,Name_Dr.,Name_Ms.,Name_Col.,Name_Capt.,Name_Rev.,Name_Major,Family_members
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.735277,0.523008,0.381594,32.204208,0.005612,0.581369,0.202020,0.144781,0.044893,0.007856,0.001122,0.002245,0.001122,0.006734,0.002245,1.904602
std,0.486592,0.836071,13.002218,1.102743,0.806057,49.693429,0.074743,0.493612,0.401733,0.352078,0.207186,0.088337,0.033501,0.047351,0.033501,0.081830,0.047351,1.613459
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,3.000000,29.881138,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000


In [ ]:
from sklearn.model_selection import train_test_split

train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test.drop(['Survived'], axis=1)

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(randome_state = 37)
xgb.fit(train_x, train_y)

[07:16:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             randome_state=37, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = xgb.predict(train_x)
pred_val = xgb.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

train mae score:  0.21807109819871656
val mae score: 0.22929467868538542


In [ ]:
pred = grid_search.predict(test_x)
submission['Survived'] = pred

In [ ]:
submission['Survived'] = round(submission['Survived'])
submission

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Titanic/submission.csv', index=False)